In [32]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup # return shortest

def scs_list(ss):
    """ Returns the list of shortest common superstrings of given
        strings, which must be the same length """
    shortest_sup = None
    sups = []
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    
    l = len(shortest_sup)  
    
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        
        if len(sup) == l:
             sups.append(sup)  # found another short superstring
    return sups  # return shortest

It's possible for there to be multiple different shortest common superstrings for the same set of input strings. Consider the input strings ABC , BCA, CAB. One shortest common superstring is ABCAB but another is BCABC and another is CABCA.

### Example 1

In [33]:
strings = ['ABC', 'BCA', 'CAB']

In [34]:
# Returns just one shortest superstring
scs(strings)

'ABCAB'

In [35]:
# Returns list of all superstrings that are tied for shorest
scs_list(strings)

['ABCAB', 'BCABC', 'CABCA']

### Example 2

In [36]:
strings = ['GAT', 'TAG', 'TCG', 'TGC', 'AAT', 'ATA']

In [37]:
# Returns just one shortest superstring
scs(strings)

'TCGATGCAATAG'

In [38]:
# Returns list of all superstrings that are tied for shorest
scs_list(strings)

['TCGATGCAATAG',
 'TCGATAGAATGC',
 'TCGAATAGATGC',
 'TGCAATCGATAG',
 'TGCAATAGATCG',
 'AATCGATAGTGC',
 'AATGCTCGATAG',
 'AATAGATCGTGC',
 'AATAGATGCTCG',
 'AATAGTCGATGC']

In [7]:
# Returns list of all superstrings that are tied for shorest
scs_list(strings)

['AATAGATCGTGC',
 'AATAGATGCTCG',
 'AATAGTCGATGC',
 'AATCGATAGTGC',
 'AATGCTCGATAG',
 'TCGAATAGATGC',
 'TCGATAGAATGC',
 'TCGATGCAATAG',
 'TGCAATAGATCG',
 'TGCAATCGATAG']

### Questions

> Question 1

What is the length of the shortest common superstring of the following strings?

CCT, CTT, TGC, TGG, GAT, ATT

In [39]:
ss = ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT']

len(scs(ss))

11

> Question 2

How many different shortest common superstrings are there for the input strings given in the previous question?

In [40]:
len(scs_list(ss))

4

> Question 3

All the reads are the same length (100 bases) and are exact copies of substrings from the forward strand of the virus genome.  You don't have to worry about sequencing errors, ploidy, or reads coming from the reverse strand.

Assemble these reads using one of the approaches discussed, such as greedy shortest common superstring.  Since there are many reads, you might consider ways to make the algorithm faster, such as the one discussed in the programming assignment in the previous module.

How many As are there in the full, assembled genome?

Hint: the virus genome you are assembling is exactly 15,894 bases long

In [41]:
def readFASTQ(filename):    
    f = open(filename)
    reads = []
    while True:
        h = f.readline()
        if len(h) == 0:
            break
        sequence = f.readline().rstrip()
        _ = f.readline()
        quality = f.readline().rstrip()
        reads.append(sequence)
        
    return reads

In [43]:
files = './data/ads1_week4_reads.fq'
reads = readFASTQ(files)

In [45]:
def pick_maximal_overlap(reads, k):
    """ Return a pair of reads from the list with a
        maximal suffix/prefix overlap >= k.  Returns
        overlap length 0 if there are no such overlaps."""
    reada, readb = None, None
    best_olen = 0
    for a, b in itertools.permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > best_olen:
            reada, readb = a, b
            best_olen = olen
    return reada, readb, best_olen

In [46]:
def greedy_scs(reads, k):
    """ Greedy shortest-common-superstring merge.
        Repeat until no edges (overlaps of length >= k)
        remain. """
    read_a, read_b, olen = pick_maximal_overlap(reads, k)
    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        read_a, read_b, olen = pick_maximal_overlap(reads, k)
    return ''.join(reads)

In [48]:
%%time
ss = greedy_scs(reads, 10)

In [ ]:
i = 0
for s in ss:
    if s == 'A':
        i +=1
print('# As: ', i)

> Question 4

How many Ts are there in the full, assembled genome from the previous question?

In [ ]:
i = 0
for s in ss:
    if s == 'T':
        i +=1
print('# Ts: ', i)